# TP Recommandation

In [1]:
# %matplotlib tk
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import csv
import pandas

Préparation des données
Récupérez l'archive http://files.grouplens.org/datasets/movielens/ml-100k.zip, sauvegardez-la dans le répertoire /donnees de votre machine, et décompressez-la.

Le jeu de données contient 100,000 notes (entre 1 et 5) données par 943 utilisateurs sur 1682 films.

Nous utiliserons pour ce TP:

Le fichier u.data qui contient la liste des différentes notes données par les différents utilisateurs aux différents films
Le fichier u.item qui fournit des informations sur les films concernés (titre, année de sortie, genre,...)
Les données contenues dans u.data pourront être lues avec la fonction np.loadtxt (http://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html)

In [2]:
data = np.loadtxt('ml-100k/u.data', dtype='int')
data.shape

(100000, 4)

In [3]:
X = sp.coo_matrix((data[:,2], (data[:,0], data[:, 1])))

In [4]:
X = X.tolil()

In [2]:
f = open("ml-100k/u.item",'r', encoding = 'latin_1')
reader = csv.reader(f, delimiter='|')
item_data = []
for row in reader:
    item_data += [row]
item_data = np.array(item_data)

In [3]:
item_data

array([['1', 'Toy Story (1995)', '01-Jan-1995', ..., '0', '0', '0'],
       ['2', 'GoldenEye (1995)', '01-Jan-1995', ..., '1', '0', '0'],
       ['3', 'Four Rooms (1995)', '01-Jan-1995', ..., '1', '0', '0'],
       ...,
       ['1680', 'Sliding Doors (1998)', '01-Jan-1998', ..., '0', '0',
        '0'],
       ['1681', 'You So Crazy (1994)', '01-Jan-1994', ..., '0', '0', '0'],
       ['1682', 'Scream of Stone (Schrei aus Stein) (1991)',
        '08-Mar-1996', ..., '0', '0', '0']], dtype='<U134')

In [92]:
titre = []
for i in range(0,len(item_data)):
    titre.append(item_data[i][1])


print(titre[:10])

['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', 'Get Shorty (1995)', 'Copycat (1995)', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Twelve Monkeys (1995)', 'Babe (1995)', 'Dead Man Walking (1995)', 'Richard III (1995)']


# Fonctions utiles

In [50]:
def prod(x,y):
    (n,p) = x.shape
    if n ==1 :
        return x.todense().dot(y.todense().T)
    else:
        return x.todense().T.dot(y.todense())


In [9]:
def sim_cos(x,y):
    num = prod(x,y)
    den1 = np.sqrt(prod(x,x))
    den2 = np.sqrt(prod(y,y))
    if den1 * den2 == 0:  # div. par zero
        return 0
    else:
        res = num / (den1 * den2) 
        return np.asscalar(res)  

In [10]:
def sim_corr(x,y):
    x = np.array(x.todense()).squeeze()
    y = np.array(y.todense()).squeeze()
    C = np.corrcoef([x,y])
    return C[0,1]

Calculez (sous la forme d'un tableau bidimensionnel) la similarité cosinus entre les 5 premiers films de la liste et affichez-le.

In [165]:
def mesure_cosinus(x,y): #la fonction todense non reconnu
  z = np.vdot(x,y)
  z = z/np.linalg.norm(x)
  z = z/np.linalg.norm(y)
  return z
    
item_data2 = [item_data[i][5:] for i in range(0,len(item_data))] #pour avoir seulement les caractéristiques
# item_data2

In [69]:
colonne = titre[:5]
tab = []


for i in range(0,5):
    col_tab = []
    a = np.array(item_data2[i],dtype=float) #sinon erreur 'a = np.array(item_data2[i],dtype=float)'
    for j in range(0,5):
        b = np.array(item_data2[j],dtype=float)
        col_tab.append(mesure_cosinus(a,b))
    tab.append(col_tab)

    
df = pandas.DataFrame(np.array(tab), columns=colonne, index = colonne)

df


,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995)
Toy Story (1995),1.000000,0.000000,0.00000,0.333333,0.000000
GoldenEye (1995),0.000000,1.000000,0.57735,0.333333,0.333333
Four Rooms (1995),0.000000,0.577350,1.00000,0.000000,0.577350
Get Shorty (1995),0.333333,0.333333,0.00000,1.000000,0.333333
Copycat (1995),0.000000,0.333333,0.57735,0.333333,1.000000


Calculez de même le tableau de similarité de corrélation entre les mêmes films et comparez le au tableau précédent

In [164]:
def sim_corr2(x,y): #la fonction todense non reconnu
    x = np.array(x.squeeze())
    y = np.array(y.squeeze())
    C = np.corrcoef([x,y])
    return C[0,1]

a = np.array(item_data2[0],dtype=float)
b = np.array(item_data2[1],dtype=float)
# sim_corr2(a,b)

In [71]:
tab2 = []

for i in range(0,5):
    col_tab2 = []
    a = np.array(item_data2[i],dtype=float) #sinon erreur 'a = np.array(item_data2[i],dtype=float)'
    for j in range(0,5):
        b = np.array(item_data2[j],dtype=float)
        col_tab2.append(sim_corr2(a,b))
    tab2.append(col_tab2)

    
df2 = pandas.DataFrame(np.array(tab2), columns=colonne, index = colonne)

df2

,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995)
Toy Story (1995),1.000000,-0.187500,-0.102062,0.208333,-0.187500
GoldenEye (1995),-0.187500,1.000000,0.544331,0.208333,0.208333
Four Rooms (1995),-0.102062,0.544331,1.000000,-0.102062,0.544331
Get Shorty (1995),0.208333,0.208333,-0.102062,1.000000,0.208333
Copycat (1995),-0.187500,0.208333,0.544331,0.208333,1.000000


Affichez maintenant la liste des dix films les plus similaires au film 'Citizen Kane (1941)', par ordre de similarité décroissante.

In [163]:
def get_indice(film):
    count = 0
    while film != item_data[count][1]:
        count = count + 1
    return int(item_data[count][0])-1 #la liste commence à 1

# get_indice('Four Rooms (1995)')

In [162]:
def get_sim_corr(titre,film):

    indexe = titre
    tab = []
    indice = get_indice(film)
    a = np.array(item_data2[indice],dtype=float)
    
    for i in range(0,len(titre)):
        b = np.array(item_data2[i],dtype=float)
        tab.append(sim_corr2(a,b))
        
    df = pandas.DataFrame(np.array(tab), columns = [film], index = indexe)
    return df

# print(get_sim_corr(titre[:5],'Four Rooms (1995)'))

In [161]:
# df3 = get_sim_corr(titre[:30],'Four Rooms (1995)')
# df3.sort_values(['Four Rooms (1995)'])[-10:][::-1]


In [157]:
def max_top_10(titre,film):
    df = get_sim_corr(titre,film)
    return df.sort_values([film])[-10:][::-1]

max_top_10(titre,'Citizen Kane (1941)')

,Citizen Kane (1941)
Scream of Stone (Schrei aus Stein) (1991),1.0
"Fille seule, La (A Single Girl) (1995)",1.0
"Remains of the Day, The (1993)",1.0
Wild Reeds (1994),1.0
Fresh (1994),1.0
Little Buddha (1993),1.0
"Portrait of a Lady, The (1996)",1.0
Searching for Bobby Fischer (1993),1.0
Last Dance (1996),1.0
"Designated Mourner, The (1997)",1.0


# Exercice 2 : profiling

On cherche à prédire la note donnée par un utilisateur i à un film j. Pour celà, on utilise la méthode des K plus proches voisins.

Affichez tout d'abord la liste des films notés par l'utilisateur 678, avec les notes associées.

In [126]:
data = np.loadtxt('ml-100k/u.data', dtype='int')
# X = sp.coo_matrix((data[:,2], (data[:,0], data[:, 1])))
# X = X.tolil()
tab = []
for x in data :
    if x[0] == 678:
        tab.append(x)

tab

[array([      678,       111,         4, 879544397]),
 array([      678,       181,         3, 879544450]),
 array([      678,         7,         4, 879544952]),
 array([      678,       924,         2, 879544397]),
 array([      678,        50,         4, 879544450]),
 array([      678,       282,         3, 879544952]),
 array([      678,        25,         2, 879544915]),
 array([      678,       287,         3, 879544397]),
 array([      678,       147,         4, 879544882]),
 array([      678,         1,         5, 879544882]),
 array([      678,      1115,         3, 879544815]),
 array([      678,       275,         2, 879544450]),
 array([      678,        15,         3, 879544449]),
 array([      678,       237,         3, 879544915]),
 array([      678,       742,         4, 879544783]),
 array([      678,       515,         4, 879544782]),
 array([      678,       276,         5, 879544952]),
 array([      678,       100,         5, 879544750]),
 array([      678,       117

La méthode des K plus proche voisins permet de prédire les notes produites par l'utilisateur à partir des notes des utilisateurs dont le profil est le plus proche.

La similarité entre utilisateurs sera basée sur les vecteurs lignes de la matrice X.

Ecrivez une fonction calcule_voisins(i,K) qui retourne la liste des K utilisateurs ayant le profil le plus similaire de celui de l'utilisateur i.

In [192]:
def get_util_info(i): #sélectionne les films et la note de l'utilisateur i
    tab = [[],[]]
    num_util = []
    for x in data :
        if x[0] == i:
            tab[0].append(x[1])
            tab[1].append(x[2])
    return tab
                       
# get_util_info(678)

In [207]:
def calcule_voisins(i,K):
    x = [] #liste des notes
    y = [] #liste des notes
    tab= []
    num_util = []
    main = get_util_info(i)
    for j in range(1,944): #on parcourt l'ensemble des utilisateurs
        other = get_util_info(j)
        num_util.append("Utilisateur "+str(j))
        for k in range(0,len(main[0])): #on parcourt les films notés par l'utilisateur
            for m in range(0,len(other[0])): #on parcourt les films notés par les autres utilisateurs
                if main[0][k] == other[0][m]: #on regarde quand l'utilisateur a noté le même film que d'autres
                    x.append(main[1][k])
                    y.append(other[1][m])
        tab.append(np.corrcoef(x,y)[0,1])
        
    df = pandas.DataFrame(tab, columns = ['Utilisateur'], index = num_util)    
    return df.sort_values(['Utilisateur'])[-K:][::-1]

voisins = calcule_voisins(678,50)
voisins



,Utilisateur
Utilisateur 1,0.266008
Utilisateur 2,0.243587
Utilisateur 5,0.182409
Utilisateur 13,0.173806
Utilisateur 8,0.173044
Utilisateur 10,0.172299
Utilisateur 7,0.172067
Utilisateur 11,0.168139
Utilisateur 14,0.167392
Utilisateur 24,0.164177


# Prédiction 1 : moyenne pondérée

Ecrivez une fonction moy_pond(i, voisins, j) qui, à partir de la liste des voisins, retourne l'estimation de la note attribuée au film j par moyenne pondérée selon la formule :

predi,j=mi+∑v∈Voisins(i)sim(i,v)(rv,j−mv)∑v∈Voisins(i)sim(i,v)
avec :

mi note moyenne de l'utilisateur i
ri,j note attribuée par l'utilisateur i au film j
sim(i,v) : similarité entre l'utilosateur i et l'utilisateur v

In [237]:
import re

def get_liste(voisins):
    liste_voisin = [] 
    for x in voisins.index:
        x = re.sub('Utilisateur ','',x)
        liste_voisin.append(int(x))
    return liste_voisin

In [227]:
def moyenne(i):
    return np.mean(get_util_info(i)[1])    

3.4285714285714284

In [252]:
def moy_pond(i, voisins, j):
    somme_num = 0
    somme_den = 0
    liste_voisin = get_liste(voisins)
    for x in liste_voisin:
        info_voisin = get_util_info(x)
        count = 0
        while (j != info_voisin[0][count] and count < len(info_voisin[0])-1) :
            count = count + 1
        R_v_j = info_voisin[1][count] #la note du voisin x sur le film j
        moyenne_voisin = moyenne(x)
        somme_num = somme_num + voisins.loc['Utilisateur '+str(x)]*(R_v_j-moyenne_voisin)
        somme_den = somme_den + voisins.loc['Utilisateur '+str(x)]
    
    return moyenne(i) + (somme_num/somme_den)
        
moy_pond(678,voisins,100)
    

Utilisateur    4.061248
dtype: float64

# Prédiction 2 : le vote

Ecrivez une fonction vote(voisins, j) qui, à partir de la liste des voisins, retourne la note la plus probable par vote majoritaire (chaque voisin vote pour sa note).

In [259]:
def vote(voisins, j):
    liste_notes = []
    liste_voisin = get_liste(voisins)
    for x in liste_voisin:
        info_voisin = get_util_info(x)
        count = 0
        while (j != info_voisin[0][count] and count < len(info_voisin[0])-1) :
            count = count + 1
        liste_notes.append(info_voisin[1][count])
        
    for i in range(2,6):
        note = 0
        max = liste_notes.count(1)
        if liste_notes.count(i) > max:
            max = liste_notes.count(i)
            note = i
    return note

vote(voisins, 100)



5

Testez ces 2 fonctions sur la liste des films de l'utilsateur 678. Affichez pour chaque film la note réelle et sa prédiction selon ces 2 méthodes.

Calculez également l'erreur moyenne et déterminez laquelle de ces méthides est la plus précise.

In [264]:
liste_film_678 = get_util_info(678)[0]
liste_note_678 = get_util_info(678)[1]

In [268]:
#Méthode 1

erreur = 0
for i in range (0,len(liste_film_678)):
    vrai_note = liste_note_678[i]
    predict = moy_pond(678, voisins, liste_film_678[i])
    erreur = erreur + (vrai_note-predict)

resultat = erreur/len(liste_film_678)

resultat


    

Utilisateur   -0.131847
dtype: float64

In [269]:
#Méthode 2

erreur = 0
for i in range (0,len(liste_film_678)):
    vrai_note = liste_note_678[i]
    predict = vote(voisins, liste_film_678[i])
    erreur = erreur + (vrai_note-predict)

resultat = erreur/len(liste_film_678)

resultat

-1.5714285714285714

Nous concluons que l'erreur en valeur absolue est plus grande avec la méthode 2.